# Example Gawain notebook
In this notebook I show how to set up, run, and plot a simple simulation using the gawain plasma physics module.


In [ ]:
import numpy as np
from gawain.main import run_gawain
from gawain.io import Reader

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

# Set up run
Here we define the simulation parameters and initial and boundary conditions.
For this simple example, I use the Sod shock tube problem. This is a 1D hydrodynamics problem, and so mhd routines are turned off.

First define the run_name and output directory, this will create a directory containing the output from the simulation.

In [ ]:
run_name = "sod_shock_tube"
output_dir = "."

Here I choose whether to run an MHD or Hydro simulation, and whether to turn on thermal conductivity and resistivty. As the Sod shock tube is a hydrodynamic problem, MHD and resistivity are turned off. I also do not turn on thermal conductivity.

In [ ]:
with_mhd = False
with_thermal_conductivity = False
with_resistivity = False

These cells define the cfl number, the total simulation time, and which time integrator and flux calculation methods are to be used. 

Currently the supported time integration methods are
- euler forward step
- 2nd order Runge-Kutta
- Leapfrog
- Predictor-Corrector

The currently supported flux calculation methods are
- Lax-Wendroff (two-step Richtmeyer form)
- Lax-Friedrichs
- HLLE with MUSCL reconstruction

For all but the simplest simulations it is strongly advised to use HLL, as Lax-Wendroff is susceptible to oscillations about sharp discontinuities and Lax-Friedrichs is very diffusive.

In [ ]:
cfl = 0.5
t_max = 0.25

In [ ]:
# "euler", "rk2", "leapfrog", "predictor-corrector"
integrator = "euler"
# "lax-wendroff", "lax-friedrichs", "hll"
fluxer = "hll"

## Define mesh

This cell defines the mesh shape (number of cells in each direction), dimensions (length of each dimension) and the number of output dumps to use.

In [ ]:
nx, ny, nz = 200, 1, 1

mesh_shape = (nx, ny, nz)

n_outputs = 100

lx, ly, lz = 1.0, 0.001, 0.001

mesh_size = (lx, ly, lz)

x = np.linspace(0.0, lx,num=nx)
y = np.linspace(0.0, ly,num=ny)
z = np.linspace(0.0, lz,num=nz)
X,Y,Z =np.meshgrid(x,y,z, indexing='ij')

## Define initial condition

The mesh information is used to create an initial condition. If this were an mhd simulation, the magnetic field initial condition would also need to be included.

In [ ]:
adiabatic_idx = 7.0/5.0

rho = np.piecewise(X, [X < 0.5, X >= 0.5], [1.0, 0.125])

pressure = np.piecewise(X, [X < 0.5, X >= 0.5], [1.0, 0.1])

mx = np.zeros(X.shape)
my = np.zeros(X.shape)
mz = np.zeros(X.shape)

e = pressure/(adiabatic_idx-1) + 0.5*mx*mx/rho

initial_condition = np.array([rho, mx, my, mz, e])

source = 0.0*np.ones(initial_condition.shape)

adiabatic_idx = 7.0/5.0

rho = np.ones(mesh_shape)

pressure = np.ones(mesh_shape)

mx = np.zeros(mesh_shape)
my = np.zeros(mesh_shape)
mz = np.zeros(mesh_shape)

e = pressure/(adiabatic_idx-1) + 0.5*mx*mx/rho

initial_condition = np.array([rho, mx, my, mz, e])

rho_s= np.zeros(mesh_shape)
mx_s= np.zeros(mesh_shape)
my_s= np.zeros(mesh_shape)
mz_s= np.zeros(mesh_shape)
e_s=np.zeros(mesh_shape)
e_s[80:120, :, :]=1.0

source = np.array([rho_s, mx_s, my_s, mz_s, e_s])

## Define boundary conditions
The available boundary conditions are
- periodic
- fixed (to the value specified in the initial condition)
- reflective

In [ ]:
boundary_conditions = ['fixed', 'periodic', 'periodic']

In [ ]:
config = {
    "run_name": run_name,
    "cfl": cfl,
    "mesh_shape": mesh_shape,
    "mesh_size": mesh_size,
    "t_max": t_max,
    "n_dumps": n_outputs,
    "initial_condition": initial_condition,
    "boundary_type": boundary_conditions,
    "adi_idx": adiabatic_idx,
    "integrator": integrator,
    "fluxer": fluxer,
    "output_dir": output_dir,
    "with_mhd": with_mhd,
    "source":source,
}

# Run Simulation
Combine all the above simulation parameters into a parameter dictionary. This dictionary is then fed to the run_gawain function which begins the simulation. Ensure the all keys for this dictionary are defined, and ensure the names are spelt correctly.

In [ ]:
run_gawain(config)

# Plot Results
One can create simple plots to visualise the results using the Reader object 

In [ ]:
data = Reader(run_name)

In [ ]:
data.variables

In [ ]:
data.plot('density', timesteps=[0,10,20,50,90])

One can also create animations from the raw data using the method below

In [ ]:
raw_data = data.get_data('energy')

In [ ]:
raw_data.shape

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(( 0, 200))
ax.set_ylim((0, 1))

line, = ax.plot([], [], lw=2)

In [ ]:
# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

In [ ]:
# animation function. This is called sequentially
def animate(i):
    x = np.linspace(0, 200, 200)
    y = raw_data[i].reshape(200,)
    line.set_data(x, y)
    return (line,)

In [ ]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, 
                               blit=True)

In [ ]:
HTML(anim.to_jshtml())